In [2]:
# Caso for trabalhar com Dashbords
#!pip install dash

  Using cached dash-2.6.2-py3-none-any.whl (9.8 MB)
  Using cached dash_core_components-2.0.0-py3-none-any.whl (3.8 kB)
  Using cached Flask_Compress-1.13-py3-none-any.whl (7.9 kB)
  Using cached dash_html_components-2.0.0-py3-none-any.whl (4.1 kB)
  Using cached dash_table-5.0.0-py3-none-any.whl (3.9 kB)


Importando as bibliotecas necessárias

In [3]:
#from dash import Dash, html, dcc
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go 
import re
import matplotlib.pyplot as plt

Lendo Dados do site do Ministério da Saúde.
Link para download dos dados: https://covid.saude.gov.br/

In [4]:
df2 = pd.read_csv("C:\Projetos\Previsão de Evolução do COVID-19\data\HIST_PAINEL_COVIDBR_14out2022\HIST_PAINEL_COVIDBR_2022_Parte1_14out2022.csv", sep=";" )
df3 = pd.read_csv("C:\Projetos\Previsão de Evolução do COVID-19\data\HIST_PAINEL_COVIDBR_14out2022\HIST_PAINEL_COVIDBR_2022_Parte2_14out2022.csv", sep=";" )
df = pd.concat([df2, df3], ignore_index=True)

In [5]:
df

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2022-01-01,52,210147125.0,22291507.0,3986,619105,49,21581668.0,90734.0,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2022-01-02,1,210147125.0,22293228.0,1721,619133,28,21581717.0,92378.0,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2022-01-03,1,210147125.0,22305078.0,11850,619209,76,21591847.0,94022.0,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2022-01-04,1,210147125.0,22323837.0,18759,619384,175,21603954.0,100499.0,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2022-01-05,1,210147125.0,22351104.0,27267,619513,129,21615473.0,116118.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1612648,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2022-10-10,41,3015268.0,839547.0,67,11831,0,NaN,NaN,1.0
1612649,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2022-10-11,41,3015268.0,839623.0,76,11831,0,NaN,NaN,1.0
1612650,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2022-10-12,41,3015268.0,1678298.0,838675,23660,11829,NaN,NaN,1.0
1612651,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2022-10-13,41,3015268.0,839703.0,-838595,11831,-11829,NaN,NaN,1.0


In [6]:
#Função para corrigir o label das Colunas
def corrige_colunas(col_names):
  return re.sub(r"[/| ]", "", col_names).lower()

df.columns = [corrige_colunas(col) for col in df.columns]

In [7]:
df_estados = df[(~df["estado"].isna()) & (df["municipio"].isna())]

In [15]:
df_estados['casosacumulado'].max()

6120384.0

Foi observado que existe para cada dia dois lançamentos de informação, um para o CODMUN=NaN e outro para o CODMUN=350000.0 e isso se repete para os demais estados. 

In [18]:
df_estados.loc[df_estados.estado == 'SP']

,regiao,estado,municipio,coduf,codmun,codregiaosaude,nomeregiaosaude,data,semanaepi,populacaotcu2019,casosacumulado,casosnovos,obitosacumulado,obitosnovos,recuperadosnovos,emacompanhamentonovos,interiormetropolitana
3620,Sudeste,SP,NaN,35,NaN,NaN,NaN,2022-01-01,52,45919049.0,4456469.0,361,155213,8,NaN,NaN,NaN
3621,Sudeste,SP,NaN,35,NaN,NaN,NaN,2022-01-02,1,45919049.0,4456581.0,112,155216,3,NaN,NaN,NaN
3622,Sudeste,SP,NaN,35,NaN,NaN,NaN,2022-01-03,1,45919049.0,4456745.0,164,155216,0,NaN,NaN,NaN
3623,Sudeste,SP,NaN,35,NaN,NaN,NaN,2022-01-04,1,45919049.0,4458689.0,1944,155242,26,NaN,NaN,NaN
3624,Sudeste,SP,NaN,35,NaN,NaN,NaN,2022-01-05,1,45919049.0,4461651.0,2962,155277,35,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368000,Sudeste,SP,NaN,35,350000.0,NaN,NaN,2022-10-10,41,NaN,199.0,0,0,0,NaN,NaN,NaN
1368001,Sudeste,SP,NaN,35,350000.0,NaN,NaN,2022-10-11,41,NaN,200.0,1,0,0,NaN,NaN,NaN
1368002,Sudeste,SP,NaN,35,350000.0,NaN,NaN,2022-10-12,41,NaN,399.0,199,0,0,NaN,NaN,NaN
1368003,Sudeste,SP,NaN,35,350000.0,NaN,NaN,2022-10-13,41,NaN,200.0,-199,0,0,NaN,NaN,NaN


In [26]:
df_estados[(df_estados.data == '2022-03-11') & (df_estados.estado == 'SP')]

,regiao,estado,municipio,coduf,codmun,codregiaosaude,nomeregiaosaude,data,semanaepi,populacaotcu2019,casosacumulado,casosnovos,obitosacumulado,obitosnovos,recuperadosnovos,emacompanhamentonovos,interiormetropolitana
3689,Sudeste,SP,NaN,35,NaN,NaN,NaN,2022-03-11,10,45919049.0,5112728.0,12183,165981,103,NaN,NaN,NaN
599179,Sudeste,SP,NaN,35,350000.0,NaN,NaN,2022-03-11,10,NaN,128.0,0,0,0,NaN,NaN,NaN


Desta forma foi utilizado os dados apenas que continha 'NaN' no atributo CODMUN.

In [65]:
px.line(df_estados.loc[(df_estados.estado == 'SP') & (df_estados.codmun.isna())], x= 'data', y= 'casosacumulado', title='Crescimento de Casos em São Paulo')

In [67]:
px.line(df_estados.loc[(df_estados.estado == 'RN') & (df_estados.codmun.isna())], x= 'data', y= 'casosacumulado', title='Crescimento de Casos no Rio Grande do Norte')

In [70]:
px.line(df_estados.loc[(df_estados.estado == 'RN') & (df_estados.codmun.isna())], x= 'data', y= 'obitosnovos', title='Óbitos diário no Rio Grande do Norte')

In [63]:
px.line(df_estados[(df_estados['codmun'].isna()) & (df_estados['estado']=='RN')], x= 'data', y= 'casosnovos', title='Novos Casos Diário do Rio Grande do Norte')

In [64]:
px.line(df_estados[(df_estados['codmun'].isna()) & (df_estados['estado']=='RJ')], x= 'data', y= 'casosnovos', title='Novos Casos Diário do Rio de Janeiro')

Buscando saber qual estado teve maior porcentagem de mortes

In [88]:
df_estados['populacaotcu2019'].unique()

array([ 1777225.,   881935.,  4144597.,   605761.,  8602865.,   845731.,
        1572866.,  7075181.,  3273227.,  9132078.,  3506853.,  4018127.,
        9557071.,  3337357.,  2298696., 14873064., 21168791.,  4018650.,
       17264943., 45919049., 11433957.,  7164788., 11377239.,  2778986.,
        3484466.,  7018354.,  3015268.,       nan])

In [110]:
Populacao_SP = df_estados.loc[df_estados['estado']=='SP']['populacaotcu2019'].max()
Mortes_SP = df_estados.loc[df_estados['estado']=='SP']['obitosacumulado'].max()
print(f'População de São Paulo: {Populacao_SP}')
print(f'Numero de Obitos em São Paulo: {Mortes_SP}')
print(f'Relação MorteXPopulação: {round((Mortes_SP/Populacao_SP),4)}%')

População de São Paulo: 45919049.0
Numero de Obitos em São Paulo: 175252
Relação MorteXPopulação: 0.0038%


In [120]:
Populacao_RN = df_estados.loc[df_estados['estado']=='RN']['populacaotcu2019'].max()
Mortes_RN = df_estados.loc[df_estados['estado']=='RN']['obitosacumulado'].max()
print(f'População do Rio Grande do Norte: {Populacao_RN}')
print(f'Numero de Obitos no Rio Grande do Norte: {Mortes_RN}')
print(f'Relação MorteXPopulação: {round((Mortes_RN/Populacao_RN),4)}%')

População do Rio Grande do Norte: 3506853.0
Numero de Obitos no Rio Grande do Norte: 8470
Relação MorteXPopulação: 0.0024%


In [121]:
df_14102022 = df_estados.loc[(df_estados['data']=='2022-10-14') & (df_estados['codmun'].isna())]

Análise de qual estado teve a maior relação de óbitos em relação a população. 

In [124]:
dicionario = {}
for i in df_14102022.index:
    dicionario.update({df_14102022['estado'][i]:df_14102022['obitosacumulado'][i]})
    

In [125]:
dicionario

{'RO': 7361,
 'AC': 2029,
 'AM': 14341,
 'RR': 2174,
 'PA': 18873,
 'AP': 2164,
 'TO': 4205,
 'MA': 10993,
 'PI': 7956,
 'CE': 27729,
 'RN': 8470,
 'PB': 10404,
 'PE': 22348,
 'AL': 7128,
 'SE': 6440,
 'BA': 30737,
 'MG': 63835,
 'ES': 14827,
 'RJ': 75758,
 'SP': 175252,
 'PR': 45374,
 'SC': 22415,
 'RS': 41153,
 'MS': 10838,
 'MT': 14950,
 'GO': 27559,
 'DF': 11831}

In [53]:
rn = df_estados.loc[(df_estados.estado == 'RN') & (df_estados.codmun.isna())]

In [56]:
rn.loc[df.data == '2022-01-01']

,regiao,estado,municipio,coduf,codmun,codregiaosaude,nomeregiaosaude,data,semanaepi,populacaotcu2019,casosacumulado,casosnovos,obitosacumulado,obitosnovos,recuperadosnovos,emacompanhamentonovos,interiormetropolitana
1991,Nordeste,RN,NaN,24,NaN,NaN,NaN,2022-01-01,52,3506853.0,387352.0,150,7572,0,NaN,NaN,NaN


In [61]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=rn['data'], y=rn['obitosacumulado'],
              name="Mortes Acumuladas", mode="lines+markers", line = {'color':'blue'})
)